In [25]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm

from tensorflow.keras.layers import Dense
from tensorflow.keras import models
from tensorflow.keras import callbacks

from sklearn.model_selection import train_test_split
from IPython.display import display

In [37]:
input_dir="../input/efficientnet_output_straight/"
dataset_dir="../input/ranzcr-clip-catheter-line-classification/"

input_df=pd.read_csv("./models/efficientnet_output.csv")
display(input_df)

,StudyInstanceUID,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,2520,2521,2522,2523,2524,2525,2526,2527,2528,2529,2530,2531,2532,2533,2534,2535,2536,2537,2538,2539,2540,2541,2542,2543,2544,2545,2546,2547,2548,2549,2550,2551,2552,2553,2554,2555,2556,2557,2558,2559
0,1.2.826.0.1.3680043.8.498.26697628953273228189...,-0.112231,0.245513,-0.033396,-0.191451,-0.199094,-0.004325,-0.083669,0.191719,-0.123753,-0.194866,-0.140194,-0.151832,-0.066314,0.459592,-0.193466,0.178913,0.025091,0.488228,-0.032680,-0.179334,-0.027792,0.047003,-0.116781,-0.053188,-0.127394,-0.234080,0.125207,0.151880,-0.154039,0.420513,-0.178386,-0.191721,-0.146882,-0.210108,-0.209314,-0.056007,-0.093566,-0.185430,-0.206313,...,-0.136301,-0.042557,0.031296,-0.078177,-0.140581,-0.189499,0.397649,0.117201,-0.065306,-0.205304,0.016094,-0.138562,0.144261,-0.225725,0.047858,0.067759,-0.031454,-0.129746,-0.057893,-0.062005,0.225690,0.178710,0.038482,-0.044136,0.027751,-0.167612,0.070643,-0.182532,0.063088,0.217482,0.190715,-0.178776,0.320377,-0.041465,-0.231193,-0.159202,0.118353,-0.183021,0.112336,0.497603
1,1.2.826.0.1.3680043.8.498.46302891597398758759...,0.067357,0.011641,0.100913,-0.112439,0.140193,0.442245,-0.244318,0.214603,0.095342,0.051628,0.421942,0.371858,-0.107535,0.420705,-0.143641,0.210064,-0.167282,-0.205380,0.096018,0.044171,-0.030589,0.347379,0.031909,0.180001,-0.051534,0.097989,-0.043427,0.143364,0.106941,0.165874,0.105836,0.170542,-0.040223,-0.223563,-0.184318,0.198835,-0.162703,-0.082894,0.002007,...,0.180552,-0.007040,0.287334,-0.100256,0.310793,0.038650,-0.024981,0.939301,0.381839,-0.147735,-0.236857,-0.100468,0.002662,-0.228927,0.044537,0.250135,-0.086726,0.092544,-0.066756,-0.038922,-0.093403,0.035720,0.142686,0.046760,0.133829,0.510843,-0.043950,-0.161542,0.430254,-0.099304,0.089758,-0.004185,-0.029220,-0.230071,-0.154650,0.121530,-0.066817,-0.139586,-0.190869,-0.071105
2,1.2.826.0.1.3680043.8.498.23819260719748494858...,-0.073804,0.213021,-0.020444,-0.081371,-0.053769,0.206744,-0.057491,0.169386,0.163935,0.445108,0.288468,0.480731,0.195124,0.329325,0.037473,0.172170,-0.147124,0.281102,0.241590,0.083157,0.156884,-0.036663,0.116032,0.306488,-0.001739,-0.143771,0.157603,0.179599,-0.175637,0.103795,-0.159293,-0.125674,-0.200435,-0.187147,-0.044244,-0.070017,-0.091799,0.127360,0.033570,...,-0.134443,-0.080158,0.024348,0.051058,0.187364,-0.155085,0.596432,-0.055658,-0.124676,-0.122165,-0.114461,0.118394,-0.118589,-0.017198,0.122819,-0.082914,-0.146477,-0.246571,-0.110141,-0.001196,-0.044032,0.222125,0.259356,0.136670,0.157020,-0.199013,0.525672,0.009088,0.718099,0.113940,0.290390,0.100370,0.369674,-0.121252,-0.178570,-0.113657,0.314377,0.027548,-0.106127,0.388178
3,1.2.826.0.1.3680043.8.498.68286643202323212801...,0.209653,0.341568,0.198626,-0.082370,-0.162807,0.436283,0.111789,-0.146370,0.390531,-0.087504,-0.123151,-0.071473,0.218949,-0.013659,-0.169848,0.107031,-0.161656,0.392880,-0.176507,-0.134083,0.551742,0.451787,-0.208027,0.772955,-0.134201,-0.230530,0.202781,-0.062631,-0.001004,0.206947,-0.075048,-0.080513,-0.176609,-0.215312,-0.154328,-0.105588,-0.084956,0.361850,0.004911,...,-0.070235,0.005716,0.069432,0.002574,0.088318,0.043166,0.646657,0.130887,0.069883,-0.178054,-0.151960,0.041531,0.192876,-0.105710,0.045457,-0.157419,-0.005191,-0.151235,-0.154077,0.234959,0.164700,0.197263,0.388851,-0.012858,0.118190,-0.109892,0.571219,0.481072,0.044466,0.426829,0.268752,-0.081703,0.260819,-0.153018,-0.242180,0.036024,0.245464,0.005475,0.015372,0.091531
4,1.2.826.0.1.3680043.8.498.10050203009225938259...,0.084045,0.073493,0.073640,0.048036,0.380339,0.306694,-0.001131,-0.050514,0.304453,0.086120,0.353393,0.086315,-0.182943,0.218872,-0.142251,0.042167,-0.203353,-0.104460,0.192745,-0.109112,0.369153,0.230854,-0.085888,0.162476,0.020465,-0.134076,-0.032471,-0.154485,0.284112,0.499852,-0.028297,-0.077725,0.030949,-0.191446,0.071007,-0.162325,0.060220,0.018586,0.332442,...,0.092647,0.159541,0.28

Wall time: 32.4 s


In [38]:
input_shape=(2560,)
encoding_dim=128

model=models.Sequential([
    Dense(encoding_dim,activation="relu",input_shape=input_shape),
    Dense(input_shape[0],activation="sigmoid")
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 128)               327808    
_________________________________________________________________
dense_9 (Dense)              (None, 2560)              330240    
Total params: 658,048
Trainable params: 658,048
Non-trainable params: 0
_________________________________________________________________


In [39]:
X_train, X_test=train_test_split(input_array)

model.compile(optimizer="adam",loss="mse")

fit_callbacks=[callbacks.EarlyStopping(monitor="val_loss",patience=5,mode="min")]

model.fit(X_train,X_train,epochs=100,shuffle=True,validation_data=(X_test,X_test),callbacks=fit_callbacks)

score=model.evaluate(X_test,X_test,verbose=0)
print(f"\nscore: {score}")

Epoch 1/100
374/706 [==============>...............] - ETA: 3s - loss: 0.0293

KeyboardInterrupt: 